In [ ]:
!pip install haversine cartopy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 110.4 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Import necessary libraries (ensure all are present)
from google.colab import drive
import pandas as pd
import numpy as np
import os
import re
from sklearn.cluster import DBSCAN
from haversine import haversine, Unit
import matplotlib.pyplot as plt
from datetime import datetime
import warnings
from typing import Dict, List, Tuple, Optional, Any, Literal

# Suppress SettingWithCopyWarning, use .loc for assignments to avoid it where possible
pd.options.mode.chained_assignment = None  # default='warn'

# Bayesian analysis libraries
try:
    import pymc as pm
    import arviz as az
    print(f"PyMC version: {pm.__version__}")
    print(f"ArviZ version: {az.__version__}")
    BAYESIAN_ENABLED = True
except ImportError:
    print("PyMC or ArviZ not found. Bayesian analysis section will be skipped.")
    print("Install with: pip install pymc arviz")
    BAYESIAN_ENABLED = False

# Mapping libraries
try:
    import cartopy.crs as ccrs
    import cartopy.feature as cfeature
    CARTOPY_AVAILABLE = True
    print("Cartopy found, map plotting enabled.")
except ImportError:
    print("Cartopy not found. Map visualization of outliers will be skipped.")
    print("Install with: conda install cartopy or pip install cartopy")
    CARTOPY_AVAILABLE = False


# --- Configuration ---
FOLDER_PATH = '/content/drive/MyDrive/Datasheets'
OUTPUT_DIR = "/content/analysis_results_split_source_bayes_map" # Updated output dir
CLUSTER_RADIUS_KM = 10
ROLLING_WINDOW_DAYS = 5
DATASET_PREFIX = 'globe'.lower() # Or 'seabass'

# Define the column name for water body source
WATER_BODY_SOURCE_COL = 'water_body_source(hes_yellow)'
TARGET_WATER_SOURCES = ['River_Stream', 'Pond_Lake'] # Case-sensitive match to data

CROATIA_BOUNDING_BOX = {
    'min_lat': 42.4,
    'max_lat': 46.5,
    'min_lon': 13.5,
    'max_lon': 19.4
}

# Bayesian Analysis Configuration
RUN_BAYESIAN_ANALYSIS = BAYESIAN_ENABLED # Only run if libraries are loaded
BAYESIAN_TARGET_KEYS = ['tube_river_stream', 'disk_pond_lake'] # Specify which results to analyze
                                            # Use ['all'] to attempt all generated keys, or specific list
BAYESIAN_N_DRAWS = 2000
BAYESIAN_N_TUNE = 2500
BAYESIAN_N_CHAINS = 4
BAYESIAN_TARGET_ACCEPT = 0.90
BAYESIAN_PPC_HDI_PROB = 0.99 # HDI probability for identifying within-cluster outliers

# --- End Configuration ---

# Mount Google Drive
# try:
#     drive.mount('/content/drive')
# except Exception as e:
#     print(f"Error mounting Google Drive: {e}")
#     pass

# Create output directory if it doesn't exist
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)
    print(f"Created output directory: {OUTPUT_DIR}")

# --- Helper Functions ---
def find_newest_dataset(folder_path: str, dataset_prefix: str) -> Optional[str]:
    """
    Finds the newest dataset file in the specified folder matching the prefix
    and a YYYYMMDD date pattern.
    """
    csv_files = []
    pattern = re.compile(rf'^{re.escape(dataset_prefix)}_(\d{{8}})\.csv$')
    try:
        if not os.path.isdir(folder_path):
            print(f"Error: Folder not found at {folder_path}")
            return None
        for filename in os.listdir(folder_path):
            match = pattern.match(filename)
            if match:
                csv_files.append((filename, int(match.group(1))))
        if not csv_files:
            print(f"No CSV files starting with '{dataset_prefix}_YYYYMMDD' found in {folder_path}")
            return None
        csv_files.sort(key=lambda item: item[1], reverse=True)
        newest_file_name = csv_files[0][0]
        print(f"Found {len(csv_files)} file(s). Using the newest: '{newest_file_name}'.")
        return os.path.join(folder_path, newest_file_name)
    except Exception as e:
        print(f"Error finding dataset files in {folder_path}: {e}")
        return None

# --- Helper Functions ---
def load_data(file_path: str) -> Optional[pd.DataFrame]:
    """
    Loads data from a CSV file into a pandas DataFrame.
    Checks for essential columns including the new water body source column.
    """
    required_columns = [
        'latitude(sample)', 'longitude(sample)', 'measured_on',
        WATER_BODY_SOURCE_COL, # Check for the specific water source column
        # Transparency columns (at least one needed)
        'transparencies:transparencydiskimagedisappearance(m)',
        'transparencies:tubeimagedisappearance(cm)',
        'transparencies:sensorturbidityntu',
        # Other (not used in core logic yet)
        # 'depth(hes_yellow)',
        # 'source_of_distance_from_water(hes_yellow)',
        # 'estimated_tube_length(hes_yellow)',
        # 'water_body_type'
    ]
    essential_cols = ['latitude(sample)', 'longitude(sample)', 'measured_on', WATER_BODY_SOURCE_COL]
    transparency_cols = [
        'transparencies:transparency disk image disappearance (m)',
        'transparencies:tube image disappearance (cm)',
        'transparencies:sensor turbidity ntu'
    ]

    try:
        df = pd.read_csv(file_path, low_memory=False)
        print(f"Successfully imported '{os.path.basename(file_path)}'. Shape: {df.shape}")

        # Check essential columns needed for filtering/grouping/splitting
        missing_essential_cols = [col for col in essential_cols if col not in df.columns]
        if missing_essential_cols:
            print(f"Error: Missing essential columns for analysis: {missing_essential_cols}")
            return None

        # Check if *at least one* transparency column exists.
        transparency_cols_present = [col for col in transparency_cols if col in df.columns]
        if not transparency_cols_present:
             print(f"Warning: No recognized transparency columns found (disk, tube, sensor). Analysis may yield limited results.")

        # Check for other required/new columns (optional)
        missing_other_cols = [col for col in required_columns if col not in df.columns and col not in essential_cols and col not in transparency_cols]
        if missing_other_cols:
            print(f"Warning: Some expected columns are missing but not essential for core logic: {missing_other_cols}")

        return df
    except FileNotFoundError:
        print(f"Error: File not found at {file_path}")
        return None
    except pd.errors.EmptyDataError:
        print(f"Error: The file '{os.path.basename(file_path)}' is empty.")
        return None
    except Exception as e:
        print(f"Error importing '{os.path.basename(file_path)}': {e}")
        return None

# --- Step 1 ---
def filter_croatia_data(df: pd.DataFrame, bbox: Dict[str, float]) -> pd.DataFrame:
    """
    Filters the DataFrame to include only data points within the specified bounding box.
    Also ensures coordinate columns are numeric.
    """
    print("Step 1: Filtering data for Croatia...")
    df['latitude(sample)'] = pd.to_numeric(df['latitude(sample)'], errors='coerce')
    df['longitude(sample)'] = pd.to_numeric(df['longitude(sample)'], errors='coerce')

    original_count = len(df)
    df.dropna(subset=['latitude(sample)', 'longitude(sample)'], inplace=True)
    if original_count > len(df):
        print(f"  - Dropped {original_count - len(df)} rows with missing/invalid coordinates.")

    croatia_df = df[
        (df['latitude(sample)'] >= bbox['min_lat']) & (df['latitude(sample)'] <= bbox['max_lat']) &
        (df['longitude(sample)'] >= bbox['min_lon']) & (df['longitude(sample)'] <= bbox['max_lon'])
    ]
    print(f"  - Found {len(croatia_df)} data points within Croatia's bounding box.")
    return croatia_df.copy()


# --- Step 2: Determine Primary Measurement ---
def determine_primary_measurement(df: pd.DataFrame) -> pd.DataFrame:
    """
    Adds a 'primary_measurement_type' column based on available non-null transparency data.
    Priority: disk > tube > sensor.
    """
    print("Step 2: Determining primary measurement type for each row...")
    conditions = []
    choices = []
    disk_col = 'transparencies:transparency disk image disappearance (m)'
    tube_col = 'transparencies:tube image disappearance (cm)'
    sensor_col = 'transparencies:sensor turbidity ntu'

    # Check if columns exist before attempting to use them
    if disk_col in df.columns:
        df[disk_col] = pd.to_numeric(df[disk_col], errors='coerce')
        conditions.append(df[disk_col].notna())
        choices.append('disk')
    else: # Print Statements for Debugging
        print(f"  - Debug: Disk column '{disk_col}' not found.")

    if tube_col in df.columns:
        df[tube_col] = pd.to_numeric(df[tube_col], errors='coerce')
        conditions.append(df[tube_col].notna())
        choices.append('tube')
    else:
        print(f"  - Debug: Tube column '{tube_col}' not found.")

    if sensor_col in df.columns:
        df[sensor_col] = pd.to_numeric(df[sensor_col], errors='coerce')
        conditions.append(df[sensor_col].notna())
        choices.append('sensor')
    else:
        print(f"  - Debug: Sensor column '{sensor_col}' not found.")

    # Check if conditions list is empty BEFORE calling np.select
    if not conditions:
        print("  - ERROR: No valid transparency columns found or processed. Cannot determine primary measurement.")
        # Assign 'none' to all rows if no columns were found/valid
        df['primary_measurement_type'] = 'none'
    else:
        df['primary_measurement_type'] = np.select(conditions, choices, default='none')

    counts = df['primary_measurement_type'].value_counts()
    print("  - Primary measurement type counts:")
    for Mtype, count in counts.items():
        print(f"    - {Mtype}: {count}")
    return df

# --- Step 3: Separate by Measurement Type ---
def separate_by_measurement_type(df: pd.DataFrame) -> Dict[str, pd.DataFrame]:
    """
    Separates the DataFrame into a dictionary based on 'primary_measurement_type'.
    Focuses on 'disk' and 'tube'.
    """
    print("Step 3: Separating data by primary measurement type ('disk', 'tube')...")
    measurement_dfs = {}
    target_types = ['disk', 'tube']
    for m_type in target_types:
        subset_df = df[df['primary_measurement_type'] == m_type].copy()
        if not subset_df.empty:
             measurement_dfs[m_type] = subset_df
             print(f"  - Measurement type '{m_type}': {len(subset_df)} data points")
        else:
             print(f"  - Measurement type '{m_type}': 0 data points")
    return measurement_dfs


# --- Step 4: Separate by Water Body Source ---
def separate_by_water_source(measurement_df: pd.DataFrame, measurement_label: str) -> Dict[str, pd.DataFrame]:
    """
    Separates a measurement-specific DataFrame by the WATER_BODY_SOURCE_COL.
    Uses safe keys (e.g., 'River_Stream' -> 'river_stream').
    """
    print(f"Step 4 [{measurement_label}]: Separating by water body source ('{WATER_BODY_SOURCE_COL}')...")
    # Ensure the source column exists and handle NaNs
    if WATER_BODY_SOURCE_COL not in measurement_df.columns:
        print(f"  - ERROR: Water body source column '{WATER_BODY_SOURCE_COL}' not found in data for {measurement_label}. Skipping separation.")
        return {}
    measurement_df[WATER_BODY_SOURCE_COL] = measurement_df[WATER_BODY_SOURCE_COL].astype(str).fillna('unknown')

    water_sources = measurement_df[WATER_BODY_SOURCE_COL].unique()
    print(f"  - Water body sources found for '{measurement_label}': {', '.join(map(str, water_sources))}")
    water_source_dfs = {}
    for water_source in water_sources:
        # Create a safe key (lowercase, replace non-alphanumeric with underscore)
        safe_water_source_label = re.sub(r'\W+', '_', str(water_source).lower())
        if not safe_water_source_label: safe_water_source_label = 'unknown'

        df_subset = measurement_df[measurement_df[WATER_BODY_SOURCE_COL] == water_source].copy()
        if not df_subset.empty:
             water_source_dfs[safe_water_source_label] = df_subset
             print(f"    - '{water_source}' (key: '{safe_water_source_label}'): {len(df_subset)} data points")
        else:
             # This case might not happen if unique() only returns present values
             pass

    return water_source_dfs


# --- Step 5: Cluster by Location ---
def cluster_by_location(
    water_source_dfs: Dict[str, pd.DataFrame],
    eps_km: float,
    measurement_label: str,
    water_source_label: str,
    output_dir: str
) -> Dict[str, pd.DataFrame]:
    """
    Clusters data points by spatial proximity. Expects single item in dict matching water_source_label.
    Logs updated labels.
    *** ADDED: Saves a CSV mapping site_id to cluster label. ***
    """
    print(f"Step 5 [{measurement_label} / {water_source_label}]: Clustering by location (eps={eps_km} km)...")
    clustered_dfs = {}
    site_id_col = 'site_id'

    def haversine_distance_sklearn(coord1, coord2):
        return haversine((coord1[0], coord1[1]), (coord2[0], coord2[1]), unit=Unit.KILOMETERS)

    if len(water_source_dfs) != 1 or list(water_source_dfs.keys())[0] != water_source_label:
         print(f"  - ERROR: cluster_by_location expected dict with single key '{water_source_label}', but got keys {list(water_source_dfs.keys())}. Aborting clustering.")
         return water_source_dfs # Return original dict to avoid breaking flow

    df = water_source_dfs[water_source_label]

    # --- Check if site_id column exists before proceeding ---
    if site_id_col not in df.columns:
        print(f"  - Warning: Column '{site_id_col}' not found. Cannot save site-to-cluster mapping CSV.")
        save_mapping_possible = False
    else:
        save_mapping_possible = True
        original_len_sid = len(df)
        df.dropna(subset=[site_id_col], inplace=True)
        if len(df) < original_len_sid:
             print(f"  - Dropped {original_len_sid - len(df)} rows with missing '{site_id_col}'.")

    if len(df) < 2:
        print(f"  - Not enough data points (< 2) for clustering. Assigning all to cluster -1 (noise).")
        df['cluster'] = -1
        clustered_dfs[water_source_label] = df
        # Attempt to save mapping even if clustering isn't meaningful (all -1)
        if save_mapping_possible and not df.empty:
            try:
                map_df = df[[site_id_col, 'cluster']]
                safe_key = f"{measurement_label}_{water_source_label}"
                map_filename = os.path.join(output_dir, f"site_cluster_map_{safe_key}.csv")
                map_df.to_csv(map_filename, index=False)
                print(f"  - Saved site-to-cluster mapping (all noise) to: {map_filename}")
            except KeyError:
                print(f"  - Error: Could not select '{site_id_col}' or 'cluster' for saving map (not enough data).")
            except Exception as e:
                print(f"  - Error saving site-to-cluster map (not enough data): {e}")
        return clustered_dfs # Return early

    coords = df[['latitude(sample)', 'longitude(sample)']].values
    if np.isnan(coords).any():
        # Drop rows with NaN coordinates before clustering
        original_len = len(df)
        df.dropna(subset=['latitude(sample)', 'longitude(sample)'], inplace=True)
        coords = df[['latitude(sample)', 'longitude(sample)']].values
        print(f"  - Warning: Dropped {original_len - len(df)} rows with NaN coordinates before clustering.")
        if len(df) < 2:
            print(f"  - Not enough data points (< 2) after dropping NaNs. Assigning cluster -1.")
            df['cluster'] = -1
            clustered_dfs[water_source_label] = df
             # Attempt to save mapping even if clustering isn't meaningful (all -1)
            if save_mapping_possible and not df.empty:
                 try:
                     map_df = df[[site_id_col, 'cluster']]
                     safe_key = f"{measurement_label}_{water_source_label}"
                     map_filename = os.path.join(output_dir, f"site_cluster_map_{safe_key}.csv")
                     map_df.to_csv(map_filename, index=False)
                     print(f"  - Saved site-to-cluster mapping (all noise after NaN drop) to: {map_filename}")
                 except KeyError:
                     print(f"  - Error: Could not select '{site_id_col}' or 'cluster' for saving map (NaN drop).")
                 except Exception as e:
                     print(f"  - Error saving site-to-cluster map (NaN drop): {e}")
            return clustered_dfs # Return early

    # --- Perform Clustering ---
    dbscan = DBSCAN(eps=eps_km, min_samples=1, metric=haversine_distance_sklearn, algorithm='auto')
    try:
        cluster_labels = dbscan.fit_predict(coords)
        df['cluster'] = cluster_labels # Assign cluster labels
    except ValueError as ve:
         print(f"  - Error during DBSCAN clustering: {ve}. Assigning cluster -1.")
         df['cluster'] = -1
    except Exception as e:
         print(f"  - Unexpected error during DBSCAN clustering: {e}. Assigning cluster -1.")
         df['cluster'] = -1

    n_clusters = len(set(df['cluster'])) - (1 if -1 in df['cluster'].unique() else 0)
    n_noise = np.sum(df['cluster'] == -1)
    print(f"  - {n_clusters} cluster(s) found, {n_noise} noise point(s).")

    # ---Save Site ID to Cluster Mapping---
    if save_mapping_possible:
        try:
            # Select only the site ID and the newly assigned cluster
            site_cluster_map_df = df[[site_id_col, 'cluster']].copy()
            # Remove duplicates if one site_id could appear multiple times in input
            # Keep the first occurrence's cluster assignment (usually consistent within clustering step)
            site_cluster_map_df.drop_duplicates(subset=[site_id_col], keep='first', inplace=True)

            safe_key = f"{measurement_label}_{water_source_label}"
            map_filename = os.path.join(output_dir, f"site_cluster_map_{safe_key}.csv")
            site_cluster_map_df.to_csv(map_filename, index=False)
            print(f"  - Saved site-to-cluster mapping to: {map_filename}")
        except KeyError:
             print(f"  - Error: Could not select '{site_id_col}' or 'cluster' columns for saving map.")
        except Exception as e:
            print(f"  - Error saving site-to-cluster map CSV: {e}")

    clustered_dfs[water_source_label] = df
    return clustered_dfs


# --- Step 6: Calculate Rolling Stats ---
def calculate_rolling_stats(
    clustered_df: pd.DataFrame, # Receives DataFrame for one water source/measurement type
    window_days: int,
    measurement_label: Literal['disk', 'tube'],
    water_source_label: str # e.g., 'river_stream'
) -> Optional[pd.DataFrame]:
    """
    Calculates rolling stats for a specific measurement & water source DataFrame.
    Uses cluster iteration and sets time index for rolling calculations.
    Logs updated labels. The output column 'water_type_group' now contains the water source label.
    """
    print(f"Step 6 [{measurement_label} / {water_source_label}]: Calculating {window_days}-day rolling statistics...")
    window_str = f'{window_days}D'
    df = clustered_df.copy() # Work on a copy

    # Define transparency column and source name
    transparency_col, source_name = (None, None)
    if measurement_label == 'disk':
        transparency_col = 'transparencies:transparency disk image disappearance (m)'
        source_name = 'disk(m)'
    elif measurement_label == 'tube':
        transparency_col = 'transparencies:tube image disappearance (cm)'
        source_name = 'tube(cm->m)'
    else:
        print(f"  - ERROR: calculate_rolling_stats unexpected measurement type '{measurement_label}'. Skipping.")
        return None

    # Check if column exists
    if transparency_col not in df.columns:
        print(f"  - Designated transparency column '{transparency_col}' not found. Skipping stats calculation.")
        df['measurement_type_group'] = measurement_label
        df['water_type_group'] = water_source_label
        df['transparency_source'] = 'N/A'
        df['transparency_value_unified'] = np.nan
        df['rolling_avg'] = np.nan; df['rolling_var'] = np.nan
        return df

    # Convert 'measured_on' to datetime, handle errors
    df['measured_on'] = pd.to_datetime(df['measured_on'], errors='coerce')
    original_count = len(df)
    df.dropna(subset=['measured_on'], inplace=True)
    if original_count > len(df):
        print(f"  - Dropped {original_count - len(df)} rows with invalid dates.")
    if df.empty:
        print(f"  - No valid data points remain after date conversion.")
        return None

    # Sort by date
    df = df.sort_values('measured_on')

    # Prepare result columns
    df['transparency_source'] = source_name
    df['transparency_value_unified'] = np.nan # Will be in meters
    df['rolling_avg'] = np.nan
    df['rolling_var'] = np.nan

    # Calculate Unified Transparency Value (in meters)
    numeric_values = pd.to_numeric(df[transparency_col], errors='coerce')
    if measurement_label == 'disk':
        df['transparency_value_unified'] = numeric_values
    elif measurement_label == 'tube':
        df['transparency_value_unified'] = numeric_values / 100.0 # cm to m

    # Ensure cluster column is integer
    df['cluster'] = pd.to_numeric(df['cluster'], errors='coerce').fillna(-1).astype(int)

    # --- Cluster Iteration for Rolling Calculation ---
    clusters = sorted([c for c in df['cluster'].unique() if c != -1])
    print(f"  - Processing clusters using iteration: {clusters}")

    calculated_rows_count = 0 # Keep track of rows getting stats

    for cluster_id in clusters:
        cluster_mask = df['cluster'] == cluster_id
        temp_cluster_df = df.loc[cluster_mask].copy() # Create subset for the cluster

        if temp_cluster_df.empty:
            continue

        # Store original indices to assign back results correctly
        original_indices_for_cluster = temp_cluster_df.index

        # Drop rows *within the cluster* where unified value is NaN before rolling
        rows_before_dropna = len(temp_cluster_df)
        temp_cluster_df.dropna(subset=['transparency_value_unified'], inplace=True)
        rows_after_dropna = len(temp_cluster_df)
        valid_original_indices = temp_cluster_df.index # Indices that remain after dropna

        if temp_cluster_df.empty:
            continue

        # --- Perform Rolling Calculations on time index ---
        try:
            temp_cluster_df_indexed = temp_cluster_df.set_index('measured_on').sort_index()
            rolling_obj = temp_cluster_df_indexed['transparency_value_unified'].rolling(window_str, min_periods=1)
            rolling_means = rolling_obj.mean()
            rolling_vars = rolling_obj.var()

            # --- Update the main DataFrame 'df' using original indices ---
            mean_map = rolling_means.to_dict()
            var_map = rolling_vars.to_dict()
            df.loc[valid_original_indices, 'rolling_avg'] = df.loc[valid_original_indices, 'measured_on'].map(mean_map)
            df.loc[valid_original_indices, 'rolling_var'] = df.loc[valid_original_indices, 'measured_on'].map(var_map)
            calculated_rows_count += len(valid_original_indices)

        except Exception as e:
            print(f"    Cluster {int(cluster_id)}: Error during rolling calculation or update: {e}")
            if not valid_original_indices.empty:
                 df.loc[valid_original_indices, ['rolling_avg', 'rolling_var']] = np.nan

    # --- End Cluster Loop ---

    noise_mask = df['cluster'] == -1
    if noise_mask.any():
        df.loc[noise_mask, ['rolling_avg', 'rolling_var']] = np.nan

    print(f"  - Calculated rolling stats for {calculated_rows_count} out of {len(df)} data points.") # Report count

    if df.empty:
        return None
    else:
        # Add identifying columns back for easier aggregation later
        df['measurement_type_group'] = measurement_label
        df['water_type_group'] = water_source_label
        return df


# --- Step 7: Visualize Results ---
def visualize_results(result_dfs_dict: Dict[str, pd.DataFrame], output_dir: str):
    """
    Creates plots for each combination of measurement type and water source.
    Updates titles and filenames.
    """
    print("\nStep 7: Generating visualizations...")
    if not result_dfs_dict: print("  - No results to visualize."); return

    for key, df in result_dfs_dict.items():
        try:
             measurement_label, water_source_label = key.split('_', 1)
        except ValueError: print(f"  - Skipping visualization for malformed key: {key}"); continue

        water_source_title = water_source_label.replace('_', ' ').title() # e.g., "River Stream"
        measurement_title = measurement_label.title()
        full_title_prefix = f"{measurement_title} Data - {water_source_title}"
        safe_key = re.sub(r'\W+', '_', key)

        df['measured_on'] = pd.to_datetime(df['measured_on'], errors='coerce')
        plot_df = df[(df['cluster'] != -1) & df['rolling_avg'].notna() & df['measured_on'].notna()].copy()

        if plot_df.empty: print(f"  - [{key}]: No valid data to plot."); continue

        plot_df = plot_df.sort_values('measured_on')
        clusters = sorted(plot_df['cluster'].unique())
        source_str = plot_df['transparency_source'].iloc[0] if not plot_df.empty else 'N/A'
        y_label_avg = f'Rolling Avg Transparency (m)'; y_label_var = f'Rolling Variance (m^2)'

        # Rolling Average Plot
        plt.figure(figsize=(15, 8))
        for cluster_id in clusters:
            cluster_df = plot_df[plot_df['cluster'] == cluster_id]
            plt.scatter(cluster_df['measured_on'], cluster_df['rolling_avg'], label=f'Cluster {int(cluster_id)}', alpha=0.7, s=20)
        plt.title(f'{ROLLING_WINDOW_DAYS}-Day Rolling Average - {full_title_prefix} (Source: {source_str})')
        plt.xlabel('Date'); plt.ylabel(y_label_avg)
        plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)
        plt.grid(True, linestyle='--', alpha=0.6); plt.xticks(rotation=45)
        plt.tight_layout(rect=[0, 0, 0.85, 1])
        plot_filename_avg = os.path.join(output_dir, f"rolling_avg_{safe_key}.png")
        try: plt.savefig(plot_filename_avg); print(f"  - Saved plot: {plot_filename_avg}")
        except Exception as e: print(f"  - Error saving plot {plot_filename_avg}: {e}")
        plt.close()

        # Rolling Variance Plot
        plot_df_var = plot_df[plot_df['rolling_var'].notna()]
        if plot_df_var.empty: print(f"  - [{key}]: No data with rolling variance to plot."); continue
        clusters_var = sorted(plot_df_var['cluster'].unique())
        plt.figure(figsize=(15, 8))
        for cluster_id in clusters_var:
            cluster_df = plot_df_var[plot_df_var['cluster'] == cluster_id]
            if cluster_df.empty: continue
            plt.scatter(cluster_df['measured_on'], cluster_df['rolling_var'], label=f'Cluster {int(cluster_id)}', alpha=0.7, s=20)
        plt.title(f'{ROLLING_WINDOW_DAYS}-Day Rolling Variance - {full_title_prefix} (Source: {source_str})')
        plt.xlabel('Date'); plt.ylabel(y_label_var)
        plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)
        plt.grid(True, linestyle='--', alpha=0.6); plt.xticks(rotation=45)
        plt.tight_layout(rect=[0, 0, 0.85, 1])
        plot_filename_var = os.path.join(output_dir, f"rolling_var_{safe_key}.png")
        try: plt.savefig(plot_filename_var); print(f"  - Saved plot: {plot_filename_var}")
        except Exception as e: print(f"  - Error saving plot {plot_filename_var}: {e}")
        plt.close()


# --- Step 8: Save Results ---
def save_results_to_csv(result_dfs_dict: Dict[str, pd.DataFrame], output_dir: str):
    """
    Saves the resulting DataFrames to CSV files, named using the combined key.
    Includes the water source column.
    """
    print("\nStep 8: Saving results to CSV...")
    if not result_dfs_dict: print("  - No results to save."); return

    for key, df in result_dfs_dict.items():
        safe_key = re.sub(r'\W+', '_', key)
        csv_filename = os.path.join(output_dir, f"analysis_results_{safe_key}.csv")
        try:
            cols_to_save = [
                'latitude(sample)', 'longitude(sample)', 'measured_on',
                WATER_BODY_SOURCE_COL, # Original water source column
                'primary_measurement_type', 'cluster',
                'transparency_source', 'transparency_value_unified',
                'rolling_avg', 'rolling_var',
                'measurement_type_group', # e.g., 'disk', 'tube'
                'water_type_group',      # e.g., 'river_stream'
                'transparencies:transparency disk image disappearance (m)',
                'transparencies:tube image disappearance (cm)'
            ]

            cols_to_save_present = [col for col in cols_to_save if col in df.columns]
            df_to_save = df[cols_to_save_present]
            df_to_save.to_csv(csv_filename, index=False)
            print(f"  - Saved results to: {csv_filename}")
        except Exception as e:
            print(f"  - Error saving CSV file {csv_filename}: {e}")


# --- Main Analysis Pipeline ---
def run_croatia_water_analysis(
    df: pd.DataFrame, cluster_radius_km: float, rolling_window_days: int, output_dir: str
) -> Optional[Dict[str, pd.DataFrame]]:
    """
    Runs the complete analysis pipeline:
    1. Filters for Croatia.
    2. Determines primary measurement type (disk/tube).
    3. Separates by measurement type.
    4. Separates each measurement type by WATER_BODY_SOURCE_COL.
    5. **Filters** to keep only TARGET_WATER_SOURCES (River_Stream, Pond_Lake).
    6. Clusters each subset by location.
    7. Calculates rolling stats for each cluster.
    8. Visualizes and saves results.
    """
    print("--- Starting Croatia Water Analysis Pipeline (Disk/Tube Separated, Filtered by Water Source) ---")
    print(f"Target Water Sources: {TARGET_WATER_SOURCES}")

    croatia_df = filter_croatia_data(df, CROATIA_BOUNDING_BOX)
    if croatia_df.empty: print("Pipeline Aborted: No data found for Croatia."); return None

    croatia_df_measured = determine_primary_measurement(croatia_df)
    measurement_dfs = separate_by_measurement_type(croatia_df_measured)
    if not measurement_dfs: print("Pipeline Aborted: No data found for 'disk' or 'tube'."); return None

    all_final_results = {}
    # Convert target source names to safe keys for filtering
    target_source_keys = [re.sub(r'\W+', '_', src.lower()) for src in TARGET_WATER_SOURCES]
    print(f"Safe keys for target sources: {target_source_keys}")

    for measurement_label, measurement_df in measurement_dfs.items():
        print(f"\n--- Processing Measurement Type: {measurement_label.upper()} ---")
        water_source_dfs = separate_by_water_source(measurement_df, measurement_label)
        if not water_source_dfs: print(f"  - No water sources found for '{measurement_label}'. Skipping."); continue

        filtered_water_source_dfs = {
            key: df_subset for key, df_subset in water_source_dfs.items()
            if key in target_source_keys
        }

        if not filtered_water_source_dfs:
            print(f"  - No data found for target water sources ({', '.join(target_source_keys)}) for '{measurement_label}'. Skipping measurement type.")
            continue
        else:
             print(f"  - Focusing analysis on water sources: {list(filtered_water_source_dfs.keys())}")

        # Iterate through the *filtered* water sources
        for water_source_label, water_df in filtered_water_source_dfs.items():
            print(f"\n-- Processing Water Source: {water_source_label} (for {measurement_label}) --")
            current_water_source_dict = {water_source_label: water_df}

            clustered_water_source_dict = cluster_by_location(
                current_water_source_dict,
                cluster_radius_km,
                measurement_label,
                water_source_label,
                output_dir
            )

            clustered_df = clustered_water_source_dict.get(water_source_label)
            # Handle case where clustering itself failed and returned original dict
            if clustered_df is None or 'cluster' not in clustered_df.columns:
                 print(f"  - Clustering step did not add 'cluster' column for '{measurement_label}/{water_source_label}'. Skipping subsequent steps.")
                 continue
            if clustered_df.empty:
                 print(f"  - DataFrame empty after clustering attempt for '{measurement_label}/{water_source_label}'. Skipping.")
                 continue

            result_df = calculate_rolling_stats(clustered_df, rolling_window_days, measurement_label, water_source_label)
            if result_df is not None and not result_df.empty:

                result_key = f"{measurement_label}_{water_source_label}" # e.g., tube_river_stream
                all_final_results[result_key] = result_df
                print(f"  - Completed processing for: {result_key}")
            else:
                print(f"  - Rolling stats failed for '{measurement_label}/{water_source_label}'.")

    if not all_final_results: print("\nPipeline Warning: No final results generated for the target water sources."); return None
    visualize_results(all_final_results, output_dir)
    save_results_to_csv(all_final_results, output_dir)
    print("\n--- Rolling Statistics Analysis Pipeline Completed ---")
    return all_final_results


# --- Bayesian Analysis Function ---
def run_bayesian_variance_analysis(
    target_df: pd.DataFrame,
    target_key: str, # e.g., "tube_river_stream"
    output_dir: str,
    n_draws: int,
    n_tune: int,
    n_chains: int,
    target_accept: float,
    ppc_hdi_prob: float = 0.99
    ):
    """
    Performs Bayesian analysis on log rolling variance for a given dataset subset,
    including map visualization of outliers. Saves identified outlier measurements to CSV.
    """
    print(f"\n--- Starting Bayesian Analysis for {target_key} ---")

    # --- 1. Data Preparation ---
    print("Preparing data for Bayesian analysis...")
    # Ensure Lat/Lon/Source are present for mapping/output later
    required_cols_bayes = ['cluster', 'rolling_var', 'latitude(sample)', 'longitude(sample)', 'measured_on', WATER_BODY_SOURCE_COL]
    if not all(col in target_df.columns for col in required_cols_bayes):
         missing_c = [c for c in required_cols_bayes if c not in target_df.columns]
         print(f"  - ERROR: Missing required columns for Bayesian analysis/mapping in {target_key}: {missing_c}. Skipping.")
         return None

    analysis_data = target_df[
        (target_df['cluster'] != -1) &
        (target_df['rolling_var'].notna()) &
        (target_df['rolling_var'] > 0)
    ].copy()

    if analysis_data.empty:
        print(f"  - No valid data (cluster != -1, rolling_var > 0) found for key '{target_key}'. Skipping Bayesian analysis.")
        return None

    analysis_data['log_rolling_var'] = np.log(analysis_data['rolling_var'])

    if analysis_data['log_rolling_var'].isnull().any() or np.isinf(analysis_data['log_rolling_var']).any():
        print(f"  - Warning: Found NaN or Inf in log_rolling_var for {target_key}. Removing affected rows.")
        rows_before = len(analysis_data)
        analysis_data.replace([np.inf, -np.inf], np.nan, inplace=True)
        analysis_data.dropna(subset=['log_rolling_var'], inplace=True)
        print(f"  - Removed {rows_before - len(analysis_data)} rows with invalid log_rolling_var.")
        if analysis_data.empty:
             print(f"  - No valid log_rolling_var data remains for key '{target_key}'. Skipping Bayesian analysis.")
             return None

    unique_clusters = sorted(analysis_data['cluster'].unique())
    cluster_map = {cluster_id: i for i, cluster_id in enumerate(unique_clusters)}
    analysis_data['cluster_idx'] = analysis_data['cluster'].map(cluster_map)
    n_clusters = len(unique_clusters)

    if n_clusters == 0: print(f"  - No clusters with valid data for key '{target_key}'. Skipping."); return None
    elif n_clusters == 1: print(f"  - Only one cluster found for key '{target_key}'. Proceeding.")

    log_variance_values = analysis_data['log_rolling_var'].values
    cluster_indices = analysis_data['cluster_idx'].values
    print(f"  - Prepared {len(log_variance_values)} data points across {n_clusters} clusters: {unique_clusters}")
    print(f"  - Log variance range: Min={log_variance_values.min():.3f}, Max={log_variance_values.max():.3f}")

    # --- 2. Define Bayesian Model ---
    print("\nDefining Hierarchical Bayesian Model...")
    coords = {"cluster": unique_clusters}
    with pm.Model(coords=coords) as hierarchical_model:
        mu_population = pm.Normal('mu_population', mu=np.median(log_variance_values), sigma=5.)
        sigma_population_mu = pm.HalfCauchy('sigma_population_mu', beta=2.5)
        sigma_cluster_scale = pm.HalfCauchy('sigma_cluster_scale', beta=2.5)
        mu_cluster_offset = pm.Normal('mu_cluster_offset', mu=0., sigma=1., dims="cluster")
        mu_cluster = pm.Deterministic('mu_cluster', mu_population + mu_cluster_offset * sigma_population_mu, dims="cluster")
        sigma_cluster = pm.HalfCauchy('sigma_cluster', beta=sigma_cluster_scale, dims="cluster")
        log_var_obs = pm.Normal('log_var_obs',
                                 mu=mu_cluster[cluster_indices],
                                 sigma=sigma_cluster[cluster_indices],
                                 observed=log_variance_values)
    print("Model definition complete.")
    try:
        graph_img = pm.model_to_graphviz(hierarchical_model)
        graph_filename = os.path.join(output_dir, f"bayes_model_graph_{target_key}.png")
        graph_img.render(graph_filename, format='png', view=False, cleanup=True)
        print(f"  - Saved model graph: {graph_filename}.png")
    except Exception as e: print(f"  - Could not generate model graph: {e}")

    # --- 3. Run MCMC Sampler ---
    print("\nRunning MCMC sampler...")
    print(f"  - Settings: draws={n_draws}, tune={n_tune}, chains={n_chains}, target_accept={target_accept}")
    idata = None
    with hierarchical_model:
        try:
            idata = pm.sample(draws=n_draws, tune=n_tune, chains=n_chains, cores=1,
                               target_accept=target_accept, init='advi+adapt_diag',
                               return_inferencedata=True)
            print("Sampling complete.")
        except Exception as e:
            print(f"Error during sampling: {e}. Sampling failed.")
            # Attempt sampling again with jitter+adapt_diag as fallback
            print("Attempting fallback sampler initialization (jitter+adapt_diag)...")
            try:
                 idata = pm.sample(draws=n_draws, tune=n_tune, chains=n_chains, cores=1,
                                   target_accept=target_accept, init='jitter+adapt_diag',
                                   return_inferencedata=True)
                 print("Fallback sampling complete.")
            except Exception as e2:
                 print(f"Fallback sampling also failed: {e2}. Bayesian analysis aborted for {target_key}.")
                 return None


    # --- 4. Analyze MCMC Results ---
    if idata:
        print("\n--- Analyzing MCMC Results ---")
        # Convergence Checks
        print("\nChecking sampler convergence (R-hat, ESS)...")
        vars_for_summary = ['mu_population', 'sigma_population_mu', 'sigma_cluster_scale', 'mu_cluster', 'sigma_cluster']
        summary = az.summary(idata, var_names=vars_for_summary, hdi_prob=0.95)
        print(f"\nConvergence Summary:")
        print(summary[['mean', 'sd', 'hdi_2.5%', 'hdi_97.5%', 'ess_bulk', 'ess_tail', 'r_hat']].round(3))
        rhat_max = summary['r_hat'].max(); ess_bulk_min = summary['ess_bulk'].min(); ess_tail_min = summary['ess_tail'].min()
        warnings_convergence = []
        if rhat_max > 1.05: warnings_convergence.append(f"Max R-hat = {rhat_max:.3f} (> 1.05)")
        if ess_bulk_min < 400: warnings_convergence.append(f"Min Bulk ESS = {ess_bulk_min:.0f} (< 400)")
        if ess_tail_min < 400: warnings_convergence.append(f"Min Tail ESS = {ess_tail_min:.0f} (< 400)")
        if warnings_convergence:
            print("\nWARNINGS:")
            for w in warnings_convergence: print(f"  - {w}")
            print("  Inspect trace plots. Consider increasing tuning or reparameterizing.")
            try:
                 az.plot_trace(idata, var_names=vars_for_summary, compact=True, figsize=(15, max(10, len(vars_for_summary)*1.5)))
                 trace_filename = os.path.join(output_dir, f"bayes_trace_plots_{target_key}.png")
                 plt.savefig(trace_filename); print(f"  - Saved trace plots: {trace_filename}"); plt.close()
            except Exception as e: print(f"  - Error saving trace plots: {e}"); plt.close()
        else: print(f"\nSampler convergence diagnostics look reasonable.")

        # Posterior Analysis for Outlier Clusters
        print("\nAnalyzing cluster means (mu_cluster) for potential outliers...")
        # Forest plot
        try:
            az.plot_forest(idata, var_names=['mu_cluster', 'mu_population'], combined=True, hdi_prob=0.95, figsize=(10, max(5, n_clusters * 0.3)))
            plt.title(f'Posterior Means Log Rolling Var (95% HDI)\nClusters vs Population | Data: {target_key}')
            plt.xlabel("Mean Log(Rolling Variance)"); plt.ylabel("Parameter / Cluster ID")
            plot_filename_forest = os.path.join(output_dir, f"bayes_forest_plot_{target_key}.png")
            plt.savefig(plot_filename_forest, bbox_inches='tight'); print(f"  - Saved forest plot: {plot_filename_forest}"); plt.close()
        except Exception as e: print(f"Error plotting/saving forest plot: {e}"); plt.close()

        # Delta plot analysis
        try:
            posterior = idata.posterior
            delta_mu = posterior['mu_cluster'] - posterior['mu_population']
            idata.posterior['delta_mu'] = delta_mu.assign_coords(cluster=idata.posterior['mu_cluster'].coords['cluster'])
            print("\nSummary of Deviations (delta_mu = mu_cluster - mu_population):")
            delta_summary = az.summary(idata, var_names=['delta_mu'], hdi_prob=0.95)
            print(delta_summary[['mean', 'sd', 'hdi_2.5%', 'hdi_97.5%']].round(3))
            potential_outlier_clusters = []
            for cluster_coord in idata.posterior['delta_mu']['cluster'].values:
                 # Ensure we handle potential missing keys if summary fails
                 idx_label = f'delta_mu[{cluster_coord}]'
                 if idx_label in delta_summary.index:
                     delta_cluster_summary = delta_summary.loc[idx_label]
                     hdi_low = delta_cluster_summary['hdi_2.5%']; hdi_high = delta_cluster_summary['hdi_97.5%']
                     if not (hdi_low <= 0 <= hdi_high):
                          mean_delta = delta_cluster_summary['mean']
                          direction = "higher" if mean_delta > 0 else "lower"
                          potential_outlier_clusters.append({'id': cluster_coord, 'direction': direction, 'mean_delta': mean_delta, 'hdi': f"[{hdi_low:.3f}, {hdi_high:.3f}]"})
                 else: print(f"Warning: Could not find summary stats for delta_mu cluster {cluster_coord}")

            if potential_outlier_clusters:
                 print("\nPotential Outlier Clusters (95% HDI of delta_mu excludes 0):")
                 for outlier in potential_outlier_clusters: print(f"  - Cluster {outlier['id']}: Mean log-var tends {outlier['direction']} (Mean Delta: {outlier['mean_delta']:.3f}, HDI: {outlier['hdi']})")
                 outlier_cluster_df = pd.DataFrame(potential_outlier_clusters)
                 outlier_cluster_filename = os.path.join(output_dir, f"bayes_outlier_clusters_{target_key}.csv")
                 outlier_cluster_df.to_csv(outlier_cluster_filename, index=False); print(f"  - Saved outlier cluster summary to: {outlier_cluster_filename}")
            else: print("\nNo strong evidence for outlier clusters found (based on 95% HDI of delta_mu).")

            az.plot_forest(idata, var_names=['delta_mu'], combined=True, hdi_prob=0.95, figsize=(10, max(5, n_clusters * 0.3)), r_hat=False, ess=False)
            plt.axvline(0, color='grey', linestyle='--', label='Population Mean (Delta=0)')
            plt.title(f'Deviation of Cluster Mean Log-Var from Pop Mean (95% HDI)\nData: {target_key}')
            plt.xlabel("Delta Log(Rolling Variance) [mu_cluster - mu_population]"); plt.ylabel("Cluster ID")
            plt.legend()
            plot_filename_delta = os.path.join(output_dir, f"bayes_delta_plot_{target_key}.png")
            plt.savefig(plot_filename_delta, bbox_inches='tight'); print(f"  - Saved delta plot: {plot_filename_delta}"); plt.close()
        except Exception as e: print(f"Error during outlier cluster analysis or plotting: {e}"); plt.close()

        # --- 5. Identify Within-Cluster Data Outliers (PPC) & Map ---
        print("\n--- Identifying Within-Cluster Data Outliers (PPC) ---")
        # analysis_data should have lat/lon/source from the check at the start
        try:
            print(f"Generating posterior predictive samples (using {len(log_variance_values)} observed points)...")
            with hierarchical_model:
                 ppc = pm.sample_posterior_predictive(idata, var_names=["log_var_obs"])
            print("Posterior predictive sampling complete.")

            observed_log_vars = log_variance_values
            ppc_samples_raw = ppc.posterior_predictive['log_var_obs']
            # Handle potential issues with stacking if dimensions mismatch
            try:
                posterior_predictive_samples = ppc_samples_raw.stack(sample=("chain", "draw")).values.T
            except Exception as stack_err:
                 print(f"Error stacking PPC samples: {stack_err}. Trying alternative reshape.")
                 # Alternative if stack fails (assuming shape is chains, draws, observations)
                 n_chains_ppc, n_draws_ppc, n_obs_ppc = ppc_samples_raw.shape
                 if n_obs_ppc == len(observed_log_vars):
                     posterior_predictive_samples = ppc_samples_raw.values.reshape(-1, n_obs_ppc)
                 else:
                     print("PPC sample observation dimension mismatch. Cannot proceed with outlier check.")
                     posterior_predictive_samples = None # Flag error

            outlier_indices_ppc = []
            if posterior_predictive_samples is not None:
                print(f"Checking {len(observed_log_vars)} data points against their {ppc_hdi_prob*100:.1f}% Posterior Predictive HDI...")
                for i in range(len(observed_log_vars)):
                    obs_val = observed_log_vars[i]
                    pred_samples_for_point = posterior_predictive_samples[:, i]
                    if np.isnan(pred_samples_for_point).any(): continue
                    hdi_pred = az.hdi(pred_samples_for_point, hdi_prob=ppc_hdi_prob)
                    if obs_val < hdi_pred[0] or obs_val > hdi_pred[1]:
                        outlier_indices_ppc.append(i)

            outlier_df_ppc = pd.DataFrame() # Initialize
            if outlier_indices_ppc:
                print(f"\nFound {len(outlier_indices_ppc)} potential within-cluster data outliers (anomalous measurements/sites):")
                # Select the original rows corresponding to these outliers
                outlier_df_ppc = analysis_data.iloc[outlier_indices_ppc].copy()

                # Calculate and add the HDI interval that the observation fell outside of
                hdi_list = []
                if posterior_predictive_samples is not None:
                    for i in outlier_indices_ppc:
                         pred_samples = posterior_predictive_samples[:, i]
                         if not np.isnan(pred_samples).any():
                            hdi = az.hdi(pred_samples, hdi_prob=ppc_hdi_prob)
                            hdi_list.append(f"[{hdi[0]:.3f}, {hdi[1]:.3f}]")
                         else: hdi_list.append("NaN")
                    hdi_col_name = f'ppc_hdi_{int(ppc_hdi_prob*100)}'
                    outlier_df_ppc[hdi_col_name] = hdi_list
                else: hdi_col_name = 'ppc_hdi_unavailable' # Placeholder if PPC failed

                # Include key identifying information for the measurement/site
                cols_to_show_save = [
                    'cluster', 'measured_on', 'latitude(sample)', 'longitude(sample)',
                    WATER_BODY_SOURCE_COL, # Show the specific source
                    'rolling_var', 'log_rolling_var'
                ]
                # Add the HDI column if available
                if hdi_col_name in outlier_df_ppc.columns:
                    cols_to_show_save.append(hdi_col_name)
                # Add original transparency value if helpful and exists
                if 'transparency_value_unified' in outlier_df_ppc.columns:
                     cols_to_show_save.insert(5, 'transparency_value_unified')
                # Add any original ID column if it exists
                # id_cols = [c for c in ['id', 'site_id', 'record_id'] if c in outlier_df_ppc.columns]
                # cols_to_show_save = id_cols + cols_to_show_save

                # Ensure only existing columns are selected
                cols_to_show_save_present = [c for c in cols_to_show_save if c in outlier_df_ppc.columns]

                print(outlier_df_ppc[cols_to_show_save_present].to_string()) # Print full df content

                outlier_filename = os.path.join(output_dir, f"bayes_anomalous_sites_{target_key}.csv")
                try:
                     # Save with original index if it's meaningful, otherwise reset
                     outlier_df_ppc[cols_to_show_save_present].to_csv(outlier_filename, index=True)
                     print(f"  - Saved anomalous sites/measurements details to: {outlier_filename}")
                except Exception as e: print(f"  - Error saving anomalous sites CSV: {e}")
            else:
                print(f"\nNo within-cluster data outliers (anomalous sites) found based on {ppc_hdi_prob*100:.1f}% Posterior Predictive HDI.")

            # --- Plot Outliers on Map ---
            print("\nGenerating map of potential outliers...")
            if not CARTOPY_AVAILABLE:
                print("  - Cartopy not available, skipping map generation.")
            elif analysis_data.empty:
                 print("  - No data available for mapping.")
            else:
                try:
                    plt.figure(figsize=(10, 12))
                    ax = plt.axes(projection=ccrs.PlateCarree())
                    # Slightly larger extent for Croatia
                    ax.set_extent([CROATIA_BOUNDING_BOX['min_lon'] - 1.0, CROATIA_BOUNDING_BOX['max_lon'] + 1.0,
                                   CROATIA_BOUNDING_BOX['min_lat'] - 1.0, CROATIA_BOUNDING_BOX['max_lat'] + 1.0],
                                  crs=ccrs.PlateCarree())

                    # Add map features
                    ax.add_feature(cfeature.LAND.with_scale('10m'), facecolor='lightgray', zorder=0)
                    ax.add_feature(cfeature.OCEAN.with_scale('10m'), facecolor='lightblue', zorder=0)
                    ax.add_feature(cfeature.COASTLINE.with_scale('10m'), edgecolor='black', linewidth=0.5, zorder=2)
                    ax.add_feature(cfeature.BORDERS.with_scale('10m'), linestyle=':', edgecolor='gray', zorder=2)
                    ax.add_feature(cfeature.LAKES.with_scale('10m'), facecolor='lightblue', edgecolor='black', linewidth=0.2, zorder=1)
                    ax.add_feature(cfeature.RIVERS.with_scale('10m'), edgecolor='blue', linewidth=0.3, zorder=1)

                    gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True, linewidth=1, color='gray', alpha=0.5, linestyle='--')
                    gl.top_labels = False; gl.right_labels = False

                    # Separate non-outliers from outliers within the analysis_data
                    if not outlier_df_ppc.empty:
                        non_outlier_mask = ~analysis_data.index.isin(outlier_df_ppc.index)
                        non_outlier_df = analysis_data[non_outlier_mask]
                    else:
                        non_outlier_df = analysis_data # All points are non-outliers if none found

                    # Plot non-outliers first
                    if not non_outlier_df.empty:
                        ax.scatter(non_outlier_df['longitude(sample)'], non_outlier_df['latitude(sample)'],
                                   color='blue', s=15, alpha=0.6, label='Non-Outlier (in analysis)',
                                   transform=ccrs.PlateCarree(), zorder=3)

                    # Plot outliers on top
                    if not outlier_df_ppc.empty:
                        ax.scatter(outlier_df_ppc['longitude(sample)'], outlier_df_ppc['latitude(sample)'],
                                   color='red', s=45, alpha=0.9, marker='X', label=f'Potential Outlier ({ppc_hdi_prob*100:.0f}% PPC HDI)',
                                   transform=ccrs.PlateCarree(), zorder=4)

                    plt.title(f'Potential Within-Cluster Outliers Map\nData: {target_key}')
                    plt.legend(loc='upper right')
                    map_filename = os.path.join(output_dir, f"bayes_outlier_map_{target_key}.png")
                    plt.savefig(map_filename, bbox_inches='tight', dpi=150)
                    print(f"  - Saved outlier map to: {map_filename}")
                    plt.close()
                except Exception as e:
                    print(f"  - Error generating or saving outlier map: {e}")
                    plt.close()
            # --- End Map Plotting Section ---

            # Optional: Plot PPC
            try:
                az.plot_ppc(idata, num_pp_samples=100)
                ppc_filename = os.path.join(output_dir, f"bayes_ppc_plot_{target_key}.png")
                plt.title(f'Posterior Predictive Check | Data: {target_key}'); plt.savefig(ppc_filename)
                print(f"  - Saved PPC plot: {ppc_filename}"); plt.close()
            except Exception as e: print(f"  - Error saving PPC plot: {e}"); plt.close()

        except Exception as e:
            print(f"Error during PPC analysis or mapping: {e}")
            import traceback; traceback.print_exc()

    else: # idata is None
         print("\nMCMC sampling failed, cannot analyze results, find outliers, or generate map.")
         return None

    print(f"\n--- Bayesian Analysis Block for {target_key} Complete ---")
    return idata


PyMC version: 5.21.2
ArviZ version: 0.21.0
Cartopy found, map plotting enabled.


In [ ]:
# --- Execution ---
if __name__ == "__main__":
    print(f"Looking for datasets starting with '{DATASET_PREFIX}' in: {FOLDER_PATH}")
    print(f"Using water source column: '{WATER_BODY_SOURCE_COL}'")
    print(f"Focusing analysis on sources: {TARGET_WATER_SOURCES}")

    if DATASET_PREFIX == 'globe': effective_prefix = 'globe_water_transparency'
    elif DATASET_PREFIX == 'seabass': effective_prefix = 'seabass'
    else: print(f"Error: Unknown DATASET_PREFIX '{DATASET_PREFIX}'."); effective_prefix = None

    if effective_prefix:
        file_path = find_newest_dataset(FOLDER_PATH, effective_prefix)
        if file_path:
            master_df = load_data(file_path)
            if master_df is not None:
                # Run rolling stats analysis
                analysis_results = run_croatia_water_analysis(
                    df=master_df,
                    cluster_radius_km=CLUSTER_RADIUS_KM,
                    rolling_window_days=ROLLING_WINDOW_DAYS,
                    output_dir=OUTPUT_DIR
                )

                if analysis_results:
                    print("\nRolling statistics analysis finished successfully.")
                    # --- Run Bayesian Analysis ---
                    if RUN_BAYESIAN_ANALYSIS:
                        print("\n--- Starting Bayesian Analysis Phase ---")
                        target_keys_to_analyze = []
                        if BAYESIAN_TARGET_KEYS == ['all']:
                             target_keys_to_analyze = list(analysis_results.keys())
                        else:
                            # Ensure target keys are valid based on the filtered results
                            target_keys_to_analyze = [k for k in BAYESIAN_TARGET_KEYS if k in analysis_results]
                            missing = [k for k in BAYESIAN_TARGET_KEYS if k not in analysis_results and k != 'all']
                            if missing: print(f"Warning: Specified Bayesian target keys not generated by analysis (check source filters?): {missing}")

                        if not target_keys_to_analyze: print("No valid target keys found for Bayesian analysis (check config vs available results).")
                        else:
                            print(f"Will attempt Bayesian analysis for keys: {target_keys_to_analyze}")
                            bayes_results = {}
                            for target_key in target_keys_to_analyze:
                                print(f"\n=== Running Bayesian analysis for: {target_key} ===")
                                target_df = analysis_results[target_key]
                                idata = run_bayesian_variance_analysis(
                                    target_df=target_df, target_key=target_key, output_dir=OUTPUT_DIR,
                                    n_draws=BAYESIAN_N_DRAWS, n_tune=BAYESIAN_N_TUNE, n_chains=BAYESIAN_N_CHAINS,
                                    target_accept=BAYESIAN_TARGET_ACCEPT, ppc_hdi_prob=BAYESIAN_PPC_HDI_PROB
                                )
                                if idata: bayes_results[target_key] = idata
                                else: print(f"Bayesian analysis did not complete successfully for {target_key}.")
                        print("\n--- Bayesian Analysis Phase Complete ---")
                    else: print("\nBayesian analysis skipped (config/libs).")
                else: print("\nRolling statistics analysis generated no final data outputs for the target water sources.")
            else: print("Failed to load data.")
        else: print("No suitable dataset file found.")
    else: print("Dataset prefix not set correctly.")

    print("\n--- Full Script Execution Finished ---")

Looking for datasets starting with 'globe' in: /content/drive/MyDrive/Datasheets
Using water source column: 'water_body_source(hes_yellow)'
Focusing analysis on sources: ['River_Stream', 'Pond_Lake']
Found 1 file(s). Using the newest: 'globe_water_transparency_20250420.csv'.
Successfully imported 'globe_water_transparency_20250420.csv'. Shape: (159146, 201)
--- Starting Croatia Water Analysis Pipeline (Disk/Tube Separated, Filtered by Water Source) ---
Target Water Sources: ['River_Stream', 'Pond_Lake']
Step 1: Filtering data for Croatia...
  - Dropped 491 rows with missing/invalid coordinates.
  - Found 24885 data points within Croatia's bounding box.
Step 2: Determining primary measurement type for each row...
  - Primary measurement type counts:
    - tube: 15829
    - disk: 9056
Step 3: Separating data by primary measurement type ('disk', 'tube')...
  - Measurement type 'disk': 9056 data points
  - Measurement type 'tube': 15829 data points
Safe keys for target sources: ['river_str

Output()

Output()

ERROR:pymc.stats.convergence:There were 89 divergences after tuning. Increase `target_accept` or reparameterize.


Sampling complete.

--- Analyzing MCMC Results ---

Checking sampler convergence (R-hat, ESS)...

Convergence Summary:
                      mean     sd  hdi_2.5%  hdi_97.5%  ess_bulk  ess_tail  \
mu_population       -4.727  0.393    -5.584     -3.999    1391.0    2385.0   
sigma_population_mu  1.432  0.337     0.894      2.132    1475.0    2250.0   
sigma_cluster_scale  2.420  0.662     1.279      3.733    8346.0    5811.0   
mu_cluster[0]       -3.736  1.413    -6.557     -1.411    4400.0    5158.0   
mu_cluster[1]       -3.995  0.155    -4.303     -3.704    9318.0    5022.0   
mu_cluster[2]       -5.827  0.275    -6.369     -5.283   12060.0    5723.0   
mu_cluster[3]       -2.837  0.035    -2.907     -2.766    7618.0    6340.0   
mu_cluster[4]       -5.663  0.393    -6.429     -4.893   11049.0    5650.0   
mu_cluster[8]       -3.602  0.288    -4.188     -3.058   11519.0    5485.0   
mu_cluster[9]       -5.733  0.086    -5.910     -5.572    7525.0    5989.0   
mu_cluster[13]      -4.

...

Output()

Sampling complete.

--- Analyzing MCMC Results ---

Checking sampler convergence (R-hat, ESS)...

Convergence Summary:
                      mean     sd  hdi_2.5%  hdi_97.5%  ess_bulk  ess_tail  \
mu_population       -4.347  0.990    -6.429     -2.407    2785.0    3098.0   
sigma_population_mu  2.010  0.934     0.456      3.951    2670.0    2553.0   
sigma_cluster_scale  2.852  1.349     0.831      5.540    8103.0    5831.0   
mu_cluster[0]       -5.792  0.705    -7.166     -4.355    9043.0    6650.0   
mu_cluster[1]       -5.534  1.134    -7.692     -3.239    7100.0    5351.0   
mu_cluster[2]       -2.715  1.042    -5.019     -0.842    5147.0    3335.0   
mu_cluster[3]       -3.679  1.259    -6.177     -1.201    8210.0    6132.0   
mu_cluster[5]       -5.527  0.938    -7.368     -3.635    7897.0    5896.0   
mu_cluster[6]       -2.828  1.239    -5.291     -0.439    5725.0    4681.0   
sigma_cluster[0]     2.118  0.631     1.190      3.374    9076.0    4736.0   
sigma_cluster[1]     2.

Output()

  - Saved delta plot: /content/analysis_results_split_source_bayes_map/bayes_delta_plot_disk_pond_lake.png

--- Identifying Within-Cluster Data Outliers (PPC) ---
Generating posterior predictive samples (using 39 observed points)...


Posterior predictive sampling complete.
Checking 39 data points against their 99.0% Posterior Predictive HDI...

No within-cluster data outliers (anomalous sites) found based on 99.0% Posterior Predictive HDI.

Generating map of potential outliers...
  - Saved outlier map to: /content/analysis_results_split_source_bayes_map/bayes_outlier_map_disk_pond_lake.png
  - Error saving PPC plot: `data` argument must have the group "posterior_predictive" for ppcplot

--- Bayesian Analysis Block for disk_pond_lake Complete ---

--- Bayesian Analysis Phase Complete ---

--- Full Script Execution Finished ---
